In [1]:
import tensorflow as tf

# Определение модели
model = tf.keras.Sequential()

# Добавление слоя с размером входных данных 1024 и функцией активации ReLU
model.add(tf.keras.layers.Dense(64, input_dim=1024, activation='relu'))

# Добавление скрытого слоя с функцией активации ReLU
model.add(tf.keras.layers.Dense(32, activation='relu'))

# Добавление выходного слоя с функцией активации sigmoid для бинарной классификации
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Компиляция модели с выбором функции потерь, оптимизатора и метрики
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Вывод структуры модели в консоль
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                65600     
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 67,713
Trainable params: 67,713
Non-trainable params: 0
_________________________________________________________________


In [2]:
import os
print(os.getcwd())

/content


In [3]:
import pandas as pd
set_directory = os.chdir("/content/data")
data = pd.read_csv(os.listdir(set_directory)[0])
for root, dirs, files in os.walk(set_directory):
  # Загрузка первого датафрейма из CSV файла
  for file in files:
    dftmp = pd.read_csv(file)
    for col_name in dftmp.columns:
      print(col_name)
    # Переименование колонки
    df = df.rename(columns={'old_name': 'new_name'})

    # Вывод DataFrame
    print(df)

  # Объединение датафреймов по ключевой колонке 'smiles'
  data = pd.merge(data, dftmp, on='smiles', how='outer')

  # Вывод результата объединения
  print(data)


FileNotFoundError: ignored

In [4]:
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 36.4 MB/s eta 0:00:00


In [5]:
!pip install mordred

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.0 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176723 sha256=386047c95c11f443dc42a4ba8e876be63064022d420f2ee5b6c1089e62f525a6
  Stored in directory: /root/.cache/pip/wheels/a7/4f/b8/d4c6591f6ac944aaced7865b349477695f662388ad958743c7
Successfully built mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


In [ ]:
import numpy as np
from rdkit import Chem
from mordred import Calculator, descriptors

data = pd.read_csv('Carcinogenicity.csv')

calc = Calculator(descriptors, ignore_3D=True)

descriptors = []

for smiles in data['smiles']:
    mol = Chem.MolFromSmiles(smiles)
    desc = calc(mol)
    descriptors.append(desc)

descriptors = np.array(descriptors)
data['descriptors'] = list(descriptors)

data.to_csv('dataset_with_descriptors.csv', index=False)



In [ ]:
print(len(data['descriptors'][0]))

3


In [6]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [7]:

# загрузка датасета из CSV файла
data = pd.read_csv('gdrive/MyDrive/tox21.csv')


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Преобразование дескрипторов веществ в numpy массив
X = np.array(data['descriptors'].tolist())

# Преобразование токсичности в бинарные метки
y = np.array(data['toxicity'].tolist())

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормализация данных
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

# Определение модели
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(64, input_dim=1024, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Обучение модели
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

# Оценка качества модели на тестовой выборке
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss:.3f}, Test accuracy: {accuracy:.3f}')
